In [9]:
import numpy as np
import pandas as pd
import os

from bokeh.io import show, output_notebook, push_notebook
output_notebook()
from bokeh.models.widgets import DateRangeSlider
from bokeh.models import ColumnDataSource, HoverTool, CustomJS
from bokeh.plotting import figure
from bokeh.layouts import row, layout, widgetbox
from bokeh.tile_providers import CARTODBPOSITRON

Loading BokehJS ...

In [18]:
data_dir = '../boat/data/'
logs = os.listdir(data_dir)
dfs= [pd.read_csv(data_dir + fname, sep='\t') for fname in logs]
merged = pd.concat(dfs)
merged.head()
merged.set_index('datetime')

def to_WM(lat, lon):
    '''convert "normal" WGS84 coordinates to the web mercator coordinates that bokeh uses
    code copied from 
    https://gis.stackexchange.com/questions/247871/convert-gps-coordinates-to-web-mercator-epsg3857-using-python-pyproj
    '''
    
    # derived from the Java version explained here: http://wiki.openstreetmap.org/wiki/Mercator
    RADIUS = 6378137.0 # in meters on the equator

    lat2 = np.log(np.tan(np.pi / 4 + np.radians(lat) / 2)) * RADIUS
    lon2 = np.radians(lon) * RADIUS
    return lat2, lon2

lat, lon = to_WM(merged['boat_lat'], merged['boat_lon'])
merged['boat_lat_WM'] = lat
merged['boat_lon_WM'] = lon

In [20]:
def default_map():
#     the bounds of Prince William Sound in WGS coords
    lo_lat, hi_lat, lo_lon, hi_lon = 60, 61.5, -149, -145.5
#     convert to web mercator
    lo_lat, lo_lon = to_WM(lo_lat, lo_lon)
    hi_lat, hi_lon = to_WM(hi_lat, hi_lon)
    p = figure(x_range=(lo_lon, hi_lon), 
           y_range=(lo_lat, hi_lat), 
           x_axis_type="mercator", 
           y_axis_type="mercator",
           tools="pan,wheel_zoom,reset,zoom_in,zoom_out",
           active_scroll='wheel_zoom')
    p.add_tile(CARTODBPOSITRON)
    
    my_hover = HoverTool()
    my_hover.tooltips = [('Time:', '@datetime')]
    p.add_tools(my_hover)
    return p

In [27]:
p = default_map()
dots = p.circle(x='boat_lon_WM', y='boat_lat_WM', source=merged)

# def update():
#     print(slider.value)
# #     p.data_source.data['x'] = 
#     push_notebook()
    
def update(source=source, slider=slider, window=None):
    data = source.data
    data['end'][0] = slider.value
    source.trigger('change')


slider = DateRangeSlider(title="Date Range: ", 
                                        start=merged['datetime'].min(), 
                                        end=merged['datetime'].max(), 
                                        value=(merged['datetime'].min(), 
                                               merged['datetime'].max()), 
                                        step=1
                                        )
slider.js_on_change('value', CustomJS.from_py_func(update))



ValueError: expected an instance of type Callback, got <function update at 0x116611d08> of type function

ValueError: expected an instance of type Callback, got <function update at 0x112f4eae8> of type function